In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import json

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


def bigkinds_search(keyword, date=time.strftime('%Y-%m-%d')):
    driver_path = r'C:\DevRoot\download\chromedriver.exe'
    driver = webdriver.Chrome(driver_path)
    
    # 홈페이지 이동
    driver.get("https://www.bigkinds.or.kr/v2/news/index.do")
    
    time.sleep(3)
    
    # 검색어 입력
    driver.find_element_by_class_name('tab-btn-inner.tab1').click()
    driver.find_element_by_id('total-search-key').clear()
    driver.find_element_by_id('total-search-key').send_keys(keyword)
    
    time.sleep(2)
    
    # 검색 날짜 범위 입력 
    
#     stdate= time.strftime('%Y-%m-01')

    for i in range(10):
        driver.find_element_by_id('search-begin-date').send_keys(Keys.BACKSPACE)
    driver.find_element_by_id('search-begin-date').send_keys(date)
    driver.find_element_by_id('search-begin-date').send_keys(Keys.RETURN)
    
    time.sleep(2)
    
    # 언론사 체크박스 클릭
    driver.find_element_by_class_name('tab2').click()
    driver.find_element_by_css_selector('ul li:nth-child(1) .form-checkbox').click()
    driver.find_element_by_css_selector('ul li:nth-child(3) .form-checkbox').click()
    
    time.sleep(2)
    
    # 상세 검색 설정
    driver.find_element_by_class_name('tab5').click()
    
    time.sleep(1)
    
    driver.find_element_by_id('search-filter-type').click()
    driver.find_element_by_id('search-filter-type').send_keys(Keys.ENTER)
    
    time.sleep(1)
    # 검색 실행
    driver.find_element_by_class_name('btn.btn-md.btn-primary.news-search-btn').click()
    
    time.sleep(3)
    
    # 100개 페이징
    driver.find_element_by_id('select2').click()
    driver.find_element_by_css_selector('#select2 option:nth-child(4)').click()
    
    source = driver.page_source
    dom = BeautifulSoup(source, 'html.parser')
    news_results = dom.select('#news-results > div.news-item')
    result_end = [
        {
            '날짜' : el.select_one('div.news-inner .cont .info p').text.strip(),
            '제목' : el.select_one('div.news-inner .cont a .title-cp span').text.strip(),
            '언론사' : el.select_one('div.news-inner .cont .info div .provider').text.strip() # 언론사가 두군데에 있네 ;;
                if el.select_one('div.news-inner .cont div.info div .provider') else el.select_one('div.info div').text.strip(),
            '기사내용' : el.select_one('div.news-inner .cont a p').text.strip()
        }

        for el in news_results
    ]
    
    return pd.DataFrame(result_end)


#### 매개변수 입력 형식 : 

bigkinds_search( 검색어(문자열), 시작날짜(yyyy-mm-dd. 문자열), ~~종료날짜(yyyy-mm-dd. 문자열)~~)
- 시작날짜 default 는 당일 날짜
- 종료날짜는 아직 구현안함 default는 오늘날짜로 할 예정

In [3]:

bigkinds_search('대만', '2021-10-15')

,날짜,제목,언론사,기사내용
0,2021/11/13,"라이더들 죽어나갈 때, 기업은 막대한 이익 끌어모았다",한겨레,중국 음식배달 플랫폼 시장은 경쟁이 꽤 치열하다. 2015년 1만5천명에 불과했던 ...
1,2021/11/13,"바이든-시진핑, 15일 첫 화상회담 개최",한겨레,조 바이든 미국 대통령과 시진핑 중국 국가주석의 첫 정상회담이 오는 15일(현지시각...
2,2021/11/13,"美中 ""15일 저녁 바이든-시진핑 화상 정상회담"" 공식 발표",한국일보,미중 양국이 조 바이든 미국 대통령과 시진핑 중국 국가주석의 화상 정상회담이 이달 ...
3,2021/11/13,시진핑 3연임 기정사실화… 2022년 ‘대관식’까지 대내외 난제 해결 숙제,세계일보,11일 중국 베이징에서 열린 중국 공산당 제19기 중앙위원회 6차 전체회의(19기 ...
4,2021/11/13,"“말할 자유를 빼앗는 악당들, 모두 뿌리째 뽑아버리자”",조선일보,송재윤의 슬픈 중국: 대륙의 자유인들 <6회> ”나랏일 얘기할 땐 좀 작게 말씀하...
5,2021/11/13,김세영 LPGA 펠리컨 챔피언십 2R 선두와 2타차 5위,한라일보,김세영(28)이 타이틀 방어에 나선 미국여자프로골프(LPGA) 투어 펠리컨 챔피언십...
6,2021/11/13,中에 절대 넘어가면 안돼…美가 꼽은 핵심 첨단 기술 5가지,중앙일보,미국과 중국의 갈등은 무역 전쟁을 넘어 기술전쟁으로 전개된 지 오래다. 조 바이든 ...
7,2021/11/13,‘5조 피해 MBI’ 구속은 4명뿐…국제사기조직에 한국은 먹잇감 [Law談-이민석],중앙일보,MBI(Mobility Beyond Imagination)는 말레이시아에 본부를 둔...
8,2021/11/13,[사설] 시진핑 1인 장기 집권 공식화한 중국,한국일보,중국공산당이 11일 19기 중앙위원회 6차 전체회의에서 ‘당 100년 분투의 중대 ...
9,2021/11/13,[기자의 시각] 마음대로 역사 바꾼 시진핑,조선일보,중국 영화감독 천카이거는 문화대혁명을 비판한 1993년 영화 ‘패왕별희’로 칸 국제...


In [5]:
bigkinds_search('기후변화')

,날짜,제목,언론사,기사내용
0,2021/11/13,‘마오리족 숄’ 걸친 文대통령…“공정무역 질서 복원”,국민일보,문재인 대통령은 지난 12일 화상으로 열린 ‘2021 아시아태평양경제협력체(APEC...
1,2021/11/13,"문 대통령, APEC 정상회의 참석…""공정한 무역 질서 복원""",경향신문,문재인 대통령이 지난 12일 화상으로 열린 ‘2021 아시아태평양경제협력체(APEC...
2,2021/11/13,협상 난항에 하루 연장된 COP26… '1.5도 사수' 막판 치열한 줄다리기,한국일보,영국 글래스고에서 열리고 있는 제26차 유엔기후변화협약 당사국총회(COP26)가 협...
3,2021/11/13,"반기문 제8대 유엔 사무총장, 경찰대학에서 특별강연",충청일보,"글로벌 리더로서 경찰의 역할, 테러 대응, 지속가능한 평화, 거버넌스 강조 경찰대학..."
4,2021/11/13,"세계인의 가장 큰 ‘걱정거리’는 코로나가 아니었다, 1위는 ○○○",한겨레,"코로나 백신 보급의 효과인가, 코로나가 불평등을 심화시킨 탓일까?빈곤과 불평등 문제..."
5,2021/11/13,"인구 7000만 영국, 국민 7명 중 1명 누적 확진자 되나",세계일보,12일(현지시간) 유엔 기후변화 대응 회의가 열리고 있는 영국 글래스고에서 환경 운...
6,2021/11/13,서민경제 회복 되나… 본예산안 첫 10조원 돌파한 전남,세계일보,전남도는 지역경제 회복과 행복시책 확대 등을 반영한 10조470억원 규모의 2022...
